# Text Generation
Adapted from code at https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

In [1]:
'''Example script to generate text from Nietzsche's writings.

At least 20 epochs are required before the generated text
starts sounding coherent.

It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.

If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import re

Using TensorFlow backend.


## Data loading

In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#path = get_file('shakespeare.txt', origin='https://github.com/karpathy/char-rnn/raw/master/data/tinyshakespeare/input.txt')
#path = get_file('pinocchio_en.txt', origin='http://www.gutenberg.org/files/500/500-0.txt')
#path = get_file('pinocchio_it.txt', origin='https://www.gutenberg.org/files/52484/52484-0.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

corpus length: 600893


## Text vectorization

In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        try:
            x[i, t, char_indices[char]] = 1
        except KeyError:
            x[i, t, char_indices[' ']] = 1
    try:
        y[i, char_indices[next_chars[i]]] = 1
    except KeyError:
        x[i, t, char_indices[' ']] = 1
print('done.')

total chars: 57
nb sequences: 200285
Vectorization...
done.


## Neural Network

In [4]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
_________________________________________________________________
activation_1 (Activation)    (None, 57)                0         
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


## Functions to generate text

In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate(model, seed, temperature):
    print()
    print('----- temperature:', temperature)

    generated = ''
    print('----- Generating with seed: "' + seed + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(seed):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]

        generated += next_char
        seed = seed[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

## Training the model

In [ ]:
# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)
    seed = text[start_index: start_index + maxlen]

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        generate(model, seed, temperature)


--------------------------------------------------
Iteration 1
Epoch 1/1
200285/200285 [==============================] - 163s - loss: 1.9902   

----- temperature: 0.2
----- Generating with seed: "aps, require their
participation in such"
 and and the spirit of the spirit of the more the stree of the complety of the stretion of the streation of the prose of the suphising and stretting in all the stretton of the most in the most in the spirit of the mistried and the strection of the man is the strengin and the streations of the stretting of the compriting and and in the prisention of the stretting and the self-the self-the self-cons

----- temperature: 0.5
----- Generating with seed: "aps, require their
participation in such"
 and a strection the most and philosophing the menster the reasing in the present, but and accustions of the consention of restrave of the pristrist
dering of the and be remare of the moralion of somical the self-lome which who that is all the stare of may of the

in that he is philosopher the wear as a
has found, e such are an
animals,
truths and sastes and forme--for exist and wisqule ayood ly duting, as a cate
omin soinal dection maworagation.


rechen to be super-anceamy for themselves in the end.


55

=

----- temperature: 1.2
----- Generating with seed: "pessimism bonae voluntatis, that not onl"
y which and cates in which fungatables--the
prevale, and must t  irwito,
as goxeweds,
and
alincity spanieacies, speco
co now of thinks fron the courde, abas itial's neidenate
partiours, an axcerrgne: and in  that always antire-everorder, and evil, nobitrlikeny morals thus as the new nearn suspates the rash what
in alleastes.cefoory (eperops doken wild
tood
he sort. but it idecty-"for an old good a

--------------------------------------------------
Iteration 9
Epoch 1/1
200285/200285 [==============================] - 160s - loss: 1.4079   

----- temperature: 0.2
----- Generating with seed: "error? the metaphysical philosophy forme"
r the standin

/home/esuli/anaconda3/envs/ta17/lib/python3.6/site-packages/ipykernel/__main__.py:4: RuntimeWarning: divide by zero encountered in log


hat his cast,
grest for the pasted. thereby mottemsing of plenity moralistic using rekers that one 

----- temperature: 1.2
----- Generating with seed: "error? the metaphysical philosophy forme"
l
us naturelie around; for the tractifieri burnarisgk; that the wor-dubloun, a blood; for insteart, raik hitherto you a actions,--one musy:el per. not in a
ravess," such a greatness,
discertainy mo.atless, in order foeich
that
antir misoneoused. that seems ive anscourated. it is too there is a high are far the
words
of gentleys 
zaw all religion.


3. former by the lacth pehis the
predact name. in

--------------------------------------------------
Iteration 10
Epoch 1/1
200285/200285 [==============================] - 160s - loss: 1.3972   

----- temperature: 0.2
----- Generating with seed: "personal advantage or the desire to give"
 and and the state of the state of the same delight and consciousnce of the soul and the soul and the state of the sense of the state of the as the problemman and

he sensuant empedifacty upon concerne the soul, as it recalless.

othe such preased doctor precoss and extent plaising expression of the existence more hadden and loving forlical?--it rejudiness of depth-more, friends! that
he inteluees schopenhauer of the souther. he wishes to the future, capable and resenges are advantaging and
expressions created
in a fullti"s that in the excriate who brad
inte

----- temperature: 1.2
----- Generating with seed: "eed:
mulier taceat in ecclesia. it was t"
he care to
would polking factation in bett the
most living of the firdeing friviablew).

12rehed in gading actuality are ideal imighing uni? germals!--far to occalter to her regard in gied kind eas: he
reword, thetous
is one, exercise cloudes fied are "remarking and bestorous germers, only tho ? whoelf being, who has sugesse is
that midiligated
to
theil capable, more, orse (but cannot feeling--eve

--------------------------------------------------
Iteration 14
Epoch 1/1
200285/200285 [=============

ts and the sense of the subving the substance of the spiritual sense--the contradiction of the self-contradice of the sentimental and senses the enough, and and fact of all the preservation of the subving the same time and interpreted to the sentimental and one the contemple of the state of the senses the sense of the truth of the sentimental sense--and in the subving the same desire of the subvin

----- temperature: 0.5
----- Generating with seed: "te opinions in
books: are not books writ"
tly prosposition, and life, and sufficient and personality in the sentimental are the so called the sentimentarist, and and produces and contemplation of the hand which one is believe, self person the former of the small of really desires that the sensation of the properse and strength that is not as a proper for the great gramment of the duce of earler, or and the belief of the processing the fac

----- temperature: 1.0
----- Generating with seed: "te opinions in
books: are not books writ"
: "knows